# Data preparation

In [1]:
URL = 'https://nnfs.io/datasets/fashion_mnist_images.zip'
FILE = 'fashion_mnist_images.zip'
FOLDER = 'examples/fashion_mnist/data/'

In [2]:
import os
import urllib
import urllib.request

In [3]:
if not os.path.isfile(FILE):
    print(f"Downloading {URL} and saving as {FILE}...")
    urllib.request.urlretrieve(URL,FILE)

    from zipfile import ZipFile
    print("Unzipping images...")
    with ZipFile(FILE) as zip_images:
        zip_images.extractall(FOLDER)

# Data loading

In [4]:
import os

DIR = 'examples/fashion_mnist'
DATA_DIR = 'examples/fashion_mnist/data'
TRAIN_DIR = 'examples/fashion_mnist/data/train'
TEST_DIR = 'examples/fashion_mnist/data/test'

labels = os.listdir(TRAIN_DIR)

files = os.listdir(TRAIN_DIR + '/0')

In [5]:
import cv2
import numpy as np
image_data = cv2.imread(TRAIN_DIR + '/7/0002.png', cv2.IMREAD_UNCHANGED)

np.set_printoptions(linewidth=200)

In [6]:
def load_mnist_dataset(dataset,path):
    
    # Scan ll the directories and create a list of labels
    labels = os.listdir(os.path.join(path, dataset))
    
    X, y = [], []
    
    for label in labels:
        for file in os.listdir(os.path.join(path, dataset, label)):
            image = cv2.imread(os.path.join(path, dataset, label, file), cv2.IMREAD_UNCHANGED)
        
            X.append(image)
            y.append(label)
            
    # Convert the data to proper numpy arrays and return
    return np.array(X), np.array(y).astype('uint8')

def create_data_mnist(path):
    # Load both sets separately
    X, y = load_mnist_dataset('train', path)
    X_test, y_test = load_mnist_dataset('test', path)
    # And return all the data
    return X, y, X_test, y_test

# Training

In [7]:
X,y,X_test,y_test = create_data_mnist(DATA_DIR)

In [8]:
# shuffle
keys = np.array(range(X.shape[0]))
np.random.shuffle(keys)
X = X[keys]
y = y[keys]

In [9]:
# flatten for dense NN, from -1 to 1
X = (X.reshape(X.shape[ 0 ], - 1 ).astype(np.float32) - 127.5 ) / 127.5
X_test = (X_test.reshape(X_test.shape[ 0 ], - 1 ).astype(np.float32) -
127.5 ) / 127.5

assert -1 <= X.min(), X.max() <= 1 # Ensuring that scaling is correct
assert -1 <= X_test.min(), X_test.max() <= 1 

In [10]:
from nn.model import *

from nn.layer import *
from nn.activation import *

from nn.optimizer import *
from nn.loss import *
from nn.accuracy import *


In [13]:
model_manual = Model()

# model_manual.add(Layer_Input(X.shape[1]))

model_manual.add(Layer_Dense(X.shape[1], 128, activation=Activation_ReLU()))
model_manual.add(Layer_Dense(128, 10, activation=Activation_Softmax()))

model_manual.set(
    loss = Loss_CategoricalCrossentropy(),
    accuracy = Accuracy_Categorical(),
    optimizer = Optimizer_Adam(decay=5e-5)
)

model_manual.train(X, y, validation_data = (X_test, y_test), epochs = 5 , batch_size = 128 , print_every = 100)

parameters = model_manual.get_parameters()

epoch: 1
step: 0, acc: 0.102, loss: 11.562 (data_loss: 11.562, reg_loss: 0.000), lr: 0.001
step: 100, acc: 0.766, loss: 1.361 (data_loss: 1.361, reg_loss: 0.000), lr: 0.0009950248756218907
step: 200, acc: 0.828, loss: 1.328 (data_loss: 1.328, reg_loss: 0.000), lr: 0.0009900990099009901
step: 300, acc: 0.844, loss: 1.514 (data_loss: 1.514, reg_loss: 0.000), lr: 0.0009852216748768474
step: 400, acc: 0.844, loss: 1.273 (data_loss: 1.273, reg_loss: 0.000), lr: 0.000980392156862745
step: 468, acc: 0.865, loss: 1.479 (data_loss: 1.479, reg_loss: 0.000), lr: 0.0009771350400625367
training, acc: 0.820, loss: 1.486 (data_loss: 1.486, reg_loss: 0.000), lr: 0.0009771350400625367
epoch: 2
step: 0, acc: 0.844, loss: 1.423 (data_loss: 1.423, reg_loss: 0.000), lr: 0.0009770873027505008
step: 100, acc: 0.805, loss: 1.366 (data_loss: 1.366, reg_loss: 0.000), lr: 0.000972337012008362
step: 200, acc: 0.828, loss: 1.363 (data_loss: 1.363, reg_loss: 0.000), lr: 0.0009676326866321544
step: 300, acc: 0.859, 

In [12]:
model = Model('basic', [X.shape[1], 128, 10])
model.train(X, y, validation_data = (X_test, y_test), epochs = 5 , batch_size = 128 , print_every = 100)
parameters = model.get_parameters()

epoch: 1
step: 0, acc: 0.109, loss: 10.810 (data_loss: 10.810, reg_loss: 0.000), lr: 0.001
step: 100, acc: 0.750, loss: 1.579 (data_loss: 1.579, reg_loss: 0.000), lr: 0.0009950248756218907
step: 200, acc: 0.805, loss: 1.327 (data_loss: 1.327, reg_loss: 0.000), lr: 0.0009900990099009901
step: 300, acc: 0.859, loss: 1.139 (data_loss: 1.139, reg_loss: 0.000), lr: 0.0009852216748768474
step: 400, acc: 0.844, loss: 1.094 (data_loss: 1.094, reg_loss: 0.000), lr: 0.000980392156862745
step: 468, acc: 0.875, loss: 1.137 (data_loss: 1.137, reg_loss: 0.000), lr: 0.0009771350400625367
training, acc: 0.819, loss: 1.358 (data_loss: 1.358, reg_loss: 0.000), lr: 0.0009771350400625367
epoch: 2
step: 0, acc: 0.844, loss: 1.557 (data_loss: 1.557, reg_loss: 0.000), lr: 0.0009770873027505008
step: 100, acc: 0.789, loss: 1.173 (data_loss: 1.173, reg_loss: 0.000), lr: 0.000972337012008362
step: 200, acc: 0.836, loss: 1.223 (data_loss: 1.223, reg_loss: 0.000), lr: 0.0009676326866321544
step: 300, acc: 0.883, 

In [ ]:
if not os.path.isfile(DIR + '/params'):
    model.save_parameters(DIR + '/params')